# *K*-mer-ology

In this notebook, we'll be taking a look at *k*-mers, minimizers, and similar subsequences used commonly for sequence analysis in bioinformatics. This first bit of code is provided for convenience for printing sequences and their constituent *k*-mers.

In [1]:
def print_seq_with_kmers(sequence, k_size):
    print("seq:", sequence)
    print("-----" + "-" * len(sequence))
    for n, kmer in get_sub_mers(sequence, k_size):
        print(f"{n:3d}: {' '*n}{kmer}")

In [2]:
def print_seq_with_minimizers(sequence, k_size, window_size):
    print("seq:", sequence)
    print("-----" + "-" * len(sequence))
    for i, minimizer, offset in get_minimizers(sequence, k_size, window_size):
        minimizer_string = ("." * offset) + minimizer + ("." * (window_size - 1 - offset))
        print(f"{i:3d}: {' '*i}{minimizer_string}")

In [3]:
def print_seq_with_spaced_seed(sequence, mask):
    print("seq:", sequence)
    print("-----" + "-" * len(sequence))
    for i, seed in get_spaced_seed(sequence, mask):
        print(f"{i:3d}: {' '*i}{seed}")

In [4]:
def print_seq_with_min_strobes(sequence, k_size, window_size):
    print("seq:", sequence)
    print("-----" + "-" * len(sequence))
    for i, anchor, j, minstrobe in get_min_strobes(sequence, k_size, window_size):
        strobemer = anchor + ("." * j) + minstrobe + ("." * (window_size - j - k_size))
        print(f"{i:3d}: {' '*i}{strobemer}")

In [5]:
shortdna = "GATTACA"
meddna = "TATTCGGCGAGACTT"
longdna = "ATATTCGGCGAGACTTGCACACGAGGCGTCGCAGATGCATCGGATCCCGA"

## *K*-mers

Fill in the following function to yield every consecutive subsequence of length `sublength`, along with its position (0-based) in the sequence. Normally we would call `sublength` here `k`, but once we've implemented this function it can also be used to generate minimizer windows and other subsequences. So we'll just use the generic term "sub-mer" to refer to *k*-mers, windows, and other subsequences.

In [6]:
def get_sub_mers(sequence, sublength):
    """yield i, submer"""
    num_submers = len(sequence) - sublength + 1
    for i in range(num_submers):
        submer = sequence[i:i+sublength]
        yield i, submer

Execute the following cells to make sure *k*-mers are computed correctly

In [7]:
print_seq_with_kmers(shortdna, 5)

seq: GATTACA
------------
  0: GATTA
  1:  ATTAC
  2:   TTACA


In [8]:
print_seq_with_kmers(meddna, 5)

seq: TATTCGGCGAGACTT
--------------------
  0: TATTC
  1:  ATTCG
  2:   TTCGG
  3:    TCGGC
  4:     CGGCG
  5:      GGCGA
  6:       GCGAG
  7:        CGAGA
  8:         GAGAC
  9:          AGACT
 10:           GACTT


In [9]:
print_seq_with_kmers(longdna, 31)

seq: ATATTCGGCGAGACTTGCACACGAGGCGTCGCAGATGCATCGGATCCCGA
-------------------------------------------------------
  0: ATATTCGGCGAGACTTGCACACGAGGCGTCG
  1:  TATTCGGCGAGACTTGCACACGAGGCGTCGC
  2:   ATTCGGCGAGACTTGCACACGAGGCGTCGCA
  3:    TTCGGCGAGACTTGCACACGAGGCGTCGCAG
  4:     TCGGCGAGACTTGCACACGAGGCGTCGCAGA
  5:      CGGCGAGACTTGCACACGAGGCGTCGCAGAT
  6:       GGCGAGACTTGCACACGAGGCGTCGCAGATG
  7:        GCGAGACTTGCACACGAGGCGTCGCAGATGC
  8:         CGAGACTTGCACACGAGGCGTCGCAGATGCA
  9:          GAGACTTGCACACGAGGCGTCGCAGATGCAT
 10:           AGACTTGCACACGAGGCGTCGCAGATGCATC
 11:            GACTTGCACACGAGGCGTCGCAGATGCATCG
 12:             ACTTGCACACGAGGCGTCGCAGATGCATCGG
 13:              CTTGCACACGAGGCGTCGCAGATGCATCGGA
 14:               TTGCACACGAGGCGTCGCAGATGCATCGGAT
 15:                TGCACACGAGGCGTCGCAGATGCATCGGATC
 16:                 GCACACGAGGCGTCGCAGATGCATCGGATCC
 17:                  CACACGAGGCGTCGCAGATGCATCGGATCCC
 18:                   ACACGAGGCGTCGCAGATGCATCGGATCCCG
 19:          

## Minimizers

Minimizers are a special class of *k*-mer. Given a window of `w` consecutive *k*-mers, the minimizer is the "smallest" or "lowest" *k*-mer in the window. Fill in the function to yield each window, its position (0-based) in the sequence, and the offset of the minimizer within the window.

> *Hint: you can use `get_sub_mers()` to generate both the windows and the k-mers within the windows.*

In [10]:
def get_minimizers(sequence, k_size, window_size):
    """yield i, minimizer, offset"""
    window_length = k_size + window_size - 1
    num_windows = len(sequence) - window_length + 1
    for i, window in get_sub_mers(sequence, window_length):
        minimizer = window[:k_size]
        offset = 0
        for j, kmer in get_sub_mers(window, k_size):
            if kmer < minimizer:
                minimizer = kmer
                offset = j
        yield i, minimizer, offset

As before, execute the following cells to check your implementation. Make sure to note the number of distinct *k*-mers in the sequence versus the number of distinct minimizers. Recall that minimizers were originally designed to permit sequence matching in less space for very large data sets.

In [11]:
print_seq_with_kmers(meddna, 6)

seq: TATTCGGCGAGACTT
--------------------
  0: TATTCG
  1:  ATTCGG
  2:   TTCGGC
  3:    TCGGCG
  4:     CGGCGA
  5:      GGCGAG
  6:       GCGAGA
  7:        CGAGAC
  8:         GAGACT
  9:          AGACTT


In [12]:
print_seq_with_minimizers(meddna, 3, 4)

seq: TATTCGGCGAGACTT
--------------------
  0: .ATT..
  1:  ATT...
  2:   ..CGG.
  3:    .CGG..
  4:     ...CGA
  5:      ..CGA.
  6:       ...AGA
  7:        ..AGA.
  8:         ...ACT
  9:          ..ACT.


In [13]:
print_seq_with_kmers(meddna, 11)

seq: TATTCGGCGAGACTT
--------------------
  0: TATTCGGCGAG
  1:  ATTCGGCGAGA
  2:   TTCGGCGAGAC
  3:    TCGGCGAGACT
  4:     CGGCGAGACTT


In [14]:
print_seq_with_minimizers(meddna, 5, 7)

seq: TATTCGGCGAGACTT
--------------------
  0: .ATTCG.....
  1:  ATTCG......
  2:   .....CGAGA.
  3:    ......AGACT
  4:     .....AGACT.


In [15]:
print_seq_with_minimizers(longdna, 15, 13)

seq: ATATTCGGCGAGACTTGCACACGAGGCGTCGCAGATGCATCGGATCCCGA
-------------------------------------------------------
  0: ............ACTTGCACACGAGGC
  1:  ...........ACTTGCACACGAGGC.
  2:   ..........ACTTGCACACGAGGC..
  3:    .........ACTTGCACACGAGGC...
  4:     ........ACTTGCACACGAGGC....
  5:      .......ACTTGCACACGAGGC.....
  6:       ............ACACGAGGCGTCGCA
  7:        ...........ACACGAGGCGTCGCA.
  8:         ..........ACACGAGGCGTCGCA..
  9:          .........ACACGAGGCGTCGCA...
 10:           ........ACACGAGGCGTCGCA....
 11:            .......ACACGAGGCGTCGCA.....
 12:             ......ACACGAGGCGTCGCA......
 13:              .....ACACGAGGCGTCGCA.......
 14:               ....ACACGAGGCGTCGCA........
 15:                ...ACACGAGGCGTCGCA.........
 16:                 ..ACACGAGGCGTCGCA..........
 17:                  .ACACGAGGCGTCGCA...........
 18:                   ACACGAGGCGTCGCA............
 19:                    .ACGAGGCGTCGCAGA...........
 20:                     ACGAGGCGTCGCA

## Spaced seeds

Spaced seeds are *k*-mers with ignored "joker" positions in fixed locations in the *k*-mer. The *weight* of the seed is the number of matching positions in the pattern, and the span is the total number of matching and ignored positions.

Fill in the following function to yield each seed sequence, including the full span of matching and ignored positions. For matching positions, show the sequence character; for joker positions, show a `.` character. As with previous functions, also yield the position (0-based) of the seed in the sequence.

Note that when using spaced seeds for sequence matching, substitutions (such as mutations or sequencing errors) in any of the joker (`.`) positions will not cause the seed match to fail; that is, the seed is robust so such variations.

In [16]:
def get_spaced_seed(sequence, mask):
    """yield i, seed"""
    span = len(mask)
    for i, spanseq in get_sub_mers(sequence, span):
        seed = list()
        for maskchar, seqchar in zip(mask, spanseq):
            if maskchar == "-":
                seed.append(".")
            else:
                seed.append(seqchar)
        yield i, "".join(seed)

In [17]:
print_seq_with_spaced_seed(shortdna, "##-#")

seq: GATTACA
------------
  0: GA.T
  1:  AT.A
  2:   TT.C
  3:    TA.A


In [18]:
print_seq_with_spaced_seed(meddna, "###-###-#")

seq: TATTCGGCGAGACTT
--------------------
  0: TAT.CGG.G
  1:  ATT.GGC.A
  2:   TTC.GCG.G
  3:    TCG.CGA.A
  4:     CGG.GAG.C
  5:      GGC.AGA.T
  6:       GCG.GAC.T


In [19]:
print_seq_with_spaced_seed(longdna, "######-###-#-#######---##")

seq: ATATTCGGCGAGACTTGCACACGAGGCGTCGCAGATGCATCGGATCCCGA
-------------------------------------------------------
  0: ATATTC.GCG.G.CTTGCAC...AG
  1:  TATTCG.CGA.A.TTGCACA...GG
  2:   ATTCGG.GAG.C.TGCACAC...GC
  3:    TTCGGC.AGA.T.GCACACG...CG
  4:     TCGGCG.GAC.T.CACACGA...GT
  5:      CGGCGA.ACT.G.ACACGAG...TC
  6:       GGCGAG.CTT.C.CACGAGG...CG
  7:        GCGAGA.TTG.A.ACGAGGC...GC
  8:         CGAGAC.TGC.C.CGAGGCG...CA
  9:          GAGACT.GCA.A.GAGGCGT...AG
 10:           AGACTT.CAC.C.AGGCGTC...GA
 11:            GACTTG.ACA.G.GGCGTCG...AT
 12:             ACTTGC.CAC.A.GCGTCGC...TG
 13:              CTTGCA.ACG.G.CGTCGCA...GC
 14:               TTGCAC.CGA.G.GTCGCAG...CA
 15:                TGCACA.GAG.C.TCGCAGA...AT
 16:                 GCACAC.AGG.G.CGCAGAT...TC
 17:                  CACACG.GGC.T.GCAGATG...CG
 18:                   ACACGA.GCG.C.CAGATGC...GG
 19:                    CACGAG.CGT.G.AGATGCA...GA
 20:                     ACGAGG.GTC.C.GATGCAT...AT
 21:                      C

## Strobemers (minstrobes)

Strobemers are coupled *k*-mers composed of a fixed *k*-mer anchor and one or more additional "randomly" distributed "strobe" *k*-mers. Minstrobes use the minimizer strategy to introduce variation into the spacing between the *k*-mers in the strobemer. Randstrobes (not shown here) use a random hash function for even more variation. In both cases, strobemers are intended to take advantage of the matching/non-matching positions benefits first introduced with spaced seeds, but without a fixed matching/non-matching structure. The authors of the strobemer approach have demonstrated modest improvements in sequence alignment accuracy and performance using strobemers as seeds.

For now, we will focus on the simplest case: minstrobes with a single anchor *k*-mer followed by a single minimizer. For this we need to specify the length of the *k*-mer and the length of the subsequent window from which the minimizer will be derived.

Fill in the following function so that for each strobemer, it yields the position, anchor *k*-mer sequence, the minstrobe, and the offset of the minstrobe within its window.

Note that, as with spaced seeds, strobemers are robust to variation present in the regions marked by `.` characters. Note that the spacing between the anchor and strobe isn't fixed. Note the step-wise behavior of the strobe that mimics the behavior we saw previously with minimizers. The use of randstrobes (not shown here) is intended to introduce additional variation in the spacing of the strobes.

In [20]:
def get_min_strobes(sequence, k_size, window_size):
    """yield i, anchor, j, minstrobe"""
    span = k_size + window_size
    for i, spanseq in get_sub_mers(sequence, span):
        anchor = spanseq[:k_size]
        window = spanseq[k_size:]
        minstrobe = window[:k_size]
        offset = 0
        for j, strobe in get_sub_mers(window, k_size):
            if strobe < minstrobe:
                minstrobe = strobe
                offset = j
        yield i, anchor, offset, minstrobe

In [21]:
print_seq_with_min_strobes(meddna, 3, 6)

seq: TATTCGGCGAGACTT
--------------------
  0: TAT.CGG..
  1:  ATT...CGA
  2:   TTC..CGA.
  3:    TCG...AGA
  4:     CGG..AGA.
  5:      GGC...ACT
  6:       GCG..ACT.


In [22]:
print_seq_with_min_strobes(longdna, 9, 15)

seq: ATATTCGGCGAGACTTGCACACGAGGCGTCGCAGATGCATCGGATCCCGA
-------------------------------------------------------
  0: ATATTCGGC...ACTTGCACA...
  1:  TATTCGGCG..ACTTGCACA....
  2:   ATTCGGCGA.ACTTGCACA.....
  3:    TTCGGCGAG......ACACGAGGC
  4:     TCGGCGAGA.....ACACGAGGC.
  5:      CGGCGAGAC....ACACGAGGC..
  6:       GGCGAGACT...ACACGAGGC...
  7:        GCGAGACTT..ACACGAGGC....
  8:         CGAGACTTG.ACACGAGGC.....
  9:          GAGACTTGCACACGAGGC......
 10:           AGACTTGCA.ACGAGGCGT.....
 11:            GACTTGCACACGAGGCGT......
 12:             ACTTGCACA..AGGCGTCGC....
 13:              CTTGCACAC.AGGCGTCGC.....
 14:               TTGCACACGAGGCGTCGC......
 15:                TGCACACGA.....CGCAGATGC.
 16:                 GCACACGAG......CAGATGCAT
 17:                  CACACGAGG......AGATGCATC
 18:                   ACACGAGGC.....AGATGCATC.
 19:                    CACGAGGCG....AGATGCATC..
 20:                     ACGAGGCGT...AGATGCATC...
 21:                      CGAGGCGTC..AGATGCATC..